## Comparing table extraction between different services

Compare Textract tables vs MS Word tables for the same reports.

1. Load textract tables

In [1]:
from borehole_tables import get_tables, save_tables
from settings import get_tables_file
import pandas as pd

docs_path = 'C:/Users/andraszeka/OneDrive - ITP (Queensland Government)/textract_result/qutsample'
#docid = '2646'
extrafolder='qutsample'
#tables = get_tables(docid, training=False, extrafolder=extrafolder)
path = 'C:\\Users\\andraszeka\OneDrive - ITP (Queensland Government)\\textract_result\\qutsample\\wondershare\\texts'
ocrservice = 'wondershare'


2. Load MS Word tables and save to file

In [2]:
import glob 

def get_ids_from_path(path, ftype):
    if 'docx' in ftype:
        glpath = path + '/*/*.' + ftype
    else:
        glpath = path + '/*.' + ftype

    ws_files = glob.glob(glpath)
    fls = []
    for f in ws_files:
        file = f.split('\\')[-1].replace('.'+ftype, '')
        if 'json' in ftype:
            file = file.replace('cr_', '').replace('_fulljson', '')

        #print(file)
        try:
            docid, file_num = file.split('_')
        except:
            continue
        docid = docid.lower()
        if docid.islower(): # some files are actually char-based, not docid, don't add these
            continue 
        fls.append([docid, file_num])
        #print(docid, ' ', file_num)
    return fls


In [3]:
ws_ids = get_ids_from_path(path, 'docx')
ts_ids = get_ids_from_path('C:/Users/andraszeka/OneDrive - ITP (Queensland Government)/textract_result/qutsample/textract/fulljson', 'json')


In [4]:
def get_word_file(docid, file_num, service):
    base_path = path = 'C:\\Users\\andraszeka\OneDrive - ITP (Queensland Government)\\textract_result\\qutsample'
    id_path = base_path + '/' + service + '/texts/' + str(docid) +'/'
    fname = str(docid) + '_' + str(pad_num(file_num)) + '.docx'
    return id_path + fname

In [5]:
def pad_num(num):
    if len(str(num)) == 2:
        num = '0' + str(num)
    elif len(str(num)) == 1:
        num = '00' + str(num)
    return num

Extract MS Word tables and save them to csv.

In [6]:
from docx import Document

for i in ws_ids:
    docid, filenum = i[0], i[1]
    try:
        wordDoc = Document(get_word_file(docid, filenum, ocrservice))
    except: #BadZipFile as e: #: BadZipFile: Bad CRC-32 for file 'word/media/image39.jpeg'
        #print(e)
        print(docid, ' ', filenum)
        continue
    tables = []
    for table in wordDoc.tables:
        df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if cell.text:
                    df[i][j] = cell.text
        tables.append(pd.DataFrame(df))
    
    folder = extrafolder + '/' + ocrservice
    tablefile = get_tables_file(docid, file_num=filenum, training=False, extrafolder=folder)
    save_tables(tables, tablefile, encoding="utf-8", header=False)
    

29695   026


In [6]:
intersect = []
for i in ws_ids:
    for j in ts_ids:
        if i[0] == j[0]:
            if i[1] == pad_num(j[1]):
                intersect.append(j)

In [7]:
len(intersect), len(ts_ids), len(ws_ids)

(212, 296, 661)

In [8]:
print(intersect)

[['14142', '1'], ['14142', '2'], ['14142', '3'], ['14142', '4'], ['14142', '5'], ['14142', '6'], ['14142', '7'], ['14142', '8'], ['14142', '9'], ['14142', '10'], ['14142', '11'], ['14142', '12'], ['14142', '13'], ['14142', '14'], ['14142', '15'], ['14142', '19'], ['14142', '22'], ['14142', '23'], ['14142', '25'], ['14142', '39'], ['1664', '1'], ['1664', '2'], ['1664', '3'], ['1664', '4'], ['1664', '5'], ['1664', '6'], ['1664', '7'], ['1664', '8'], ['1664', '9'], ['1664', '10'], ['1664', '11'], ['1664', '12'], ['1664', '13'], ['1664', '14'], ['1664', '15'], ['1664', '16'], ['1664', '17'], ['1664', '18'], ['1664', '19'], ['1664', '20'], ['1664', '21'], ['1664', '23'], ['1664', '25'], ['1664', '27'], ['167', '1'], ['167', '2'], ['1799', '1'], ['1799', '2'], ['1799', '3'], ['1799', '4'], ['1799', '5'], ['1799', '6'], ['1799', '7'], ['1799', '8'], ['1799', '9'], ['1799', '10'], ['1799', '16'], ['1799', '18'], ['1799', '20'], ['1799', '21'], ['1799', '23'], ['1799', '29'], ['21166', '1'], ['

3. Compare the tables from the three services for: same table amount, etc, and save results

In [7]:
# for each file in intersect, try to get the ts and ws table file
# if doesn't exist, note '0' tables
# if does, get tables out and count the number of tables
# save results to csv

cols = ['DocID', 'FileNum', 'TXTables', 'WSTables']

for i in intersect:
    ws_tables = None
    tx_tables = None
    num_ws = 0
    num_tx = 0
    
    docid, filenum = i[0], i[1]
    
    # first try getting ws tables
    ws_folder = extrafolder + '/' + ocrservice
    tx_folder = extrafolder + '/textract'
    try:
        ws_tables = get_tables(docid, report_num=pad_num(filenum), training=False, extrafolder=ws_folder, sep=',')
    except FileNotFoundError:
        pass
    
    try:
        tx_tables = get_tables(docid, report_num=filenum, training=False, extrafolder=tx_folder, sep='`')
    except FileNotFoundError:
        pass
    
    if ws_tables:
        num_ws = len(ws_tables)
    if tx_tables:
        num_tx = len(tx_tables)
    
    
    data = [docid, filenum, num_tx, num_ws]
    sr = pd.Series(data)
    
    df = pd.DataFrame(sr).T

    with open('tables_comp.csv', 'a', newline='') as f:
        df.to_csv(f, index=False, header=False)

C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_14142_012_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_14142_022_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_14142_025_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_14142_039_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_1664_001_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_1664_002_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_1664_004_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_

C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_53382_002_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_5992_004_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_5992_005_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_5992_009_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_63981_005_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_801_002_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_801_003_tables.csv
C:\Users\andraszeka\OneDrive - ITP (Queensland Government)\textract_result/qutsample/wondershare/tables/cr_801_

In [53]:
get_word_file('2347', '1', 'wondershare')

'C:\\Users\\andraszeka\\OneDrive - ITP (Queensland Government)\\textract_result\\qutsample/wondershare/texts/2347/2347_001.docx'